## Data Preperation
This jupyter notebook performs data preparation on KK-Box data, supplied for the following Kaggle competition:
https://www.kaggle.com/c/kkbox-churn-prediction-challenge

>"In this challenge, you are asked to predict whether a user will churn after his/her subscription expires. Specifically, we want to forecast if a user make a new service subscription transaction within 30 days after the current membership expiration date.
>
>KKBOX offers subscription based music streaming service. When users signs up for our service, users can choose to either manual renew or auto-renew the service. Users can actively cancel their membership at any time.
>
>The churn/renewal definition can be tricky due to KKBox's subscription model. Since the majority of KKBox's subscription length is 30 days, a lot of users re-subscribe every month. The key fields to determine churn/renewal are transaction date, membership expiration date, and is_cancel. Note that the is_cancel field indicates whether a user actively cancels a subscription. Subscription cancellation does not imply the user has churned. A user may cancel service subscription due to change of service plans or other reasons. The criteria of "churn" is no new valid service subscription within 30 days after the current membership expires."

In this script I take some inspiration from other users submissions, as the main focus of this project is not data preparation/wrangling but rather feature contribution extraction from the GBT model (in the “LightGBM – Feature Relationships” script).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time

path = "C:/Users/andrew.davidson/OneDrive - Concentra Consulting Limited/Documents/Projects/GBT Feature Relationships/KK Box/data/churn_comp_refresh"
train = pd.read_csv(path+'/train_v2.csv')
sample_submission = pd.read_csv(path+'/sample_submission_v2.csv')
transactions = pd.read_csv(path+'/transactions_v2.csv')
user_logs = pd.read_csv(path+'/user_logs_v2.csv')
members = pd.read_csv(path+'/members_v3.csv')


In [2]:
# Reducing data set size
# Check the current size of the dataset
mem = transactions.memory_usage(index=True).sum()
print(mem/ 1024**2," MB")

def change_datatype(df):
    '''This function will reduce the data type of each column to the minimum size available given the values'''
    int_cols = list(df.select_dtypes(include='int64').columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
    float_cols = list(df.select_dtypes(include='float').columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
        
    return df

# Apply the change_datatype function to the transactions data and check the change in memory used
transactions = change_datatype(transactions)
mem = transactions.memory_usage(index=True).sum()
print(mem/ 1024**2," MB")

98.25971221923828  MB
34.118035316467285  MB


In [3]:
# Calculate whether any discounts have been applied to users trasnactions
transactions['discount'] = (transactions['plan_list_price'] - transactions['actual_amount_paid'])/transactions['plan_list_price']
transactions['is_discount'] = transactions.discount.apply(lambda x: 1 if x > 0 else 0)

# Reformate the date columns as dates
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], format='%Y%m%d')
transactions['membership_expire_date'] = pd.to_datetime(transactions['membership_expire_date'], format='%Y%m%d')

transactions = change_datatype(transactions)

In [4]:
start = time.time()
# Groups transactions by msno (user id), and performs a variety of calculations given users prior transactions

transactions_v2 = transactions.groupby('msno').agg(transaction_count=('payment_method_id', 'count'), # Number of transactions
                        total_payment_plan_days= ('payment_plan_days','sum'), # Total number of payment plan days
                        avg_payment_plan_days= ('payment_plan_days','mean'), # Avg payment plan days per transaction
                        membership_expire_date = ('membership_expire_date', 'max'), # Last expiry date of membership
                        most_recent_transaction = ('transaction_date', 'max'), # Most recent transaction date
                        first_payment_plan = ('transaction_date', 'min'),  # First transaction date
                        plan_net_worth=('plan_list_price', 'sum'), # The total list price of the users plan history
                        mean_payment_each_transaction = ('actual_amount_paid', 'mean'), # Mean amount paid per transaction
                        total_actual_payment = ('actual_amount_paid', 'sum'), # The total amount paid over the users account history
                        normal_payment_method_id = ('payment_method_id', lambda x:x.value_counts().index[0]), # Most frequently used payment method
                        auto_renew_times = ('is_auto_renew', lambda x : sum(x==1)), # Number of times auto renewed
                        cancel_times = ('is_cancel', lambda x : sum(x==1)), # Number of times the user has historically cancelled
                        average_discount= ('discount', 'mean'), # Average discount applied across transaction history
                        biggest_discount= ('discount', 'max'), # Biggest discount recieved over all transactions
                        discount_times= ('is_discount',  lambda x : sum(x==1)) # Number of times discount has been recieved
                                                  )

end = time.time()
del transactions
print(end-start)

transactions_v2['amt_per_day'] = transactions_v2['total_actual_payment'] / transactions_v2['total_payment_plan_days']

transactions_v2.reset_index(inplace=True)
transactions_v2 = change_datatype(transactions_v2)

953.3426291942596


In [5]:
# Groups user logs by msno (user id) o calculate streaming information for users for the month preceeding the churn prediction

user_logs_v2 = user_logs.groupby('msno').agg(first_listen_date =('date', 'min'), # First listen date in the period
                                most_recent_listen_date =('date', 'max'), # Last listen date in the period
                                number_days_listened = ('date', 'count'), # Number of days listened in the period
                                num_25 = ('num_25', 'sum'), # of songs played less than 25% of the song length
                                num_50 = ('num_50', 'sum'), # of songs played between 25% to 50% of the song length
                                num_75 = ('num_75', 'sum'), # of songs played between 50% to 75% of of the song length
                                num_985 = ('num_985', 'sum'), # of songs played between 75% to 98.5% of the song length
                                num_100 = ('num_100', 'sum'), # of songs played over 98.5% of the song length
                                num_unq = ('num_unq', 'sum'), # Total number of daily unique songs played
                                total_secs = ('total_secs', 'sum')) # Total number of seconds listened

del user_logs

# Calculate the total number of song plays in the listening period
user_logs_v2["total_plays"] = user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100']

# Calculate the percentage of number of songs played within certain period
user_logs_v2['percent_25'] = user_logs_v2['num_25']/(user_logs_v2['total_plays'])
user_logs_v2['percent_50'] = user_logs_v2['num_50']/(user_logs_v2['total_plays'])
user_logs_v2['percent_100'] = (user_logs_v2['num_985']+user_logs_v2['num_100'])/(user_logs_v2['total_plays'])

# Calcualte metrics based on average X per day
user_logs_v2["avg_seconds_per_day"] = user_logs_v2["total_secs"]/user_logs_v2["number_days_listened"]
user_logs_v2["avg_plays_per_day"] = user_logs_v2["total_plays"]/user_logs_v2["number_days_listened"]
user_logs_v2["avg_unique_per_day"] = user_logs_v2["num_unq"]/user_logs_v2["number_days_listened"]

# Convert date columns
user_logs_v2['first_listen_date'] = pd.to_datetime(user_logs_v2['first_listen_date'], format = '%Y%m%d')
user_logs_v2['most_recent_listen_date'] = pd.to_datetime(user_logs_v2['most_recent_listen_date'], format = '%Y%m%d')

# drop unused variables
user_logs_v2 = user_logs_v2.drop(["num_unq"], axis = 1)
user_logs_v2.reset_index(inplace = True)
user_logs_v2 = change_datatype(user_logs_v2)

In [6]:
# Merge the aggregated user information between different tables to the churn data (train)
dataset_train = train.merge(members, on = 'msno', how = 'left').merge(transactions_v2, on = 'msno', how = 'left').merge(user_logs_v2, on = 'msno', how = 'left')

# Reduce space requirements
del transactions_v2, user_logs_v2

# Convert the date columns to date
dataset_train['registration_init_time'] = pd.to_datetime(dataset_train['registration_init_time'], format = '%Y%m%d')

# Create new date diff columns for modelling purpose:
# Days between membership expirey and the latest listen
dataset_train['day_diff_membership_expire__last_listen'] = (dataset_train['membership_expire_date'] - dataset_train['most_recent_listen_date']).astype('timedelta64[D]')
# The days between the most recent listen and the first listen (within the period)
dataset_train['day_diff_last_listen__first_listen'] = (dataset_train['most_recent_listen_date'] - dataset_train['first_listen_date']).astype('timedelta64[D]')
# Day difference between the membership expiry and the initial registration (customer tenure)
dataset_train['day_diff_membership_expire__registration'] = (dataset_train['membership_expire_date'] - dataset_train['registration_init_time']).astype('timedelta64[D]')
# Day difference between membership expirey and the day before the churn calculation
dataset_train['day_diff_membership_expire__registration'] = (dataset_train['membership_expire_date'] - dataset_train.most_recent_listen_date.max()).astype('timedelta64[D]')

# This feature was removed due to the data leakage meaning the model was "cheating"
#dataset_train['day_diff_membership_expire__last_transaction'] = (dataset_train['membership_expire_date'] - dataset_train['most_recent_transaction']).astype('timedelta64[D]')

# Fix other columns
# bd: Age (years), messy column, replaced values that are outwith the expected range with Nulls
dataset_train['bd'] = np.where((dataset_train['bd'] >100), None,dataset_train['bd'])
dataset_train['bd'] = np.where((dataset_train['bd'] <5), None,dataset_train['bd'])

# Flag for if the user has no listening data during the period
dataset_train["no_listen_flag"] = dataset_train.first_listen_date.isnull().astype(int)
# Flag for if the user has no transaction history (this shouldnt not happen but is added due to data quality issues)
dataset_train["no_transactions_flag"] = dataset_train.transaction_count.isnull().astype(int)

# remove unsured columns
dataset_train_v2 = dataset_train.drop(columns = ['msno', 'registration_init_time', 'most_recent_transaction', 'membership_expire_date', 'most_recent_listen_date', 'first_listen_date', 'first_payment_plan'])
del dataset_train
# Reset data types to minimise space requirement
dataset_train_v2 = change_datatype(dataset_train_v2)

In [7]:
# Define a functiona that replaces outliers. 
def replaceoutlier(df, cols):
    '''This function replaces outliers given a list of columns. 
    An outlier is defined as a value which is greater than the mean +- 3x standard deviation
    The replacement value is the mean +- 3x standard deviation'''
    for i in cols:
        mean, std = np.mean(dataset_train_v2[i]), np.std(dataset_train_v2[i])
        cut_off = std*3
        lower, upper = mean - cut_off, mean + cut_off
        df[i][df[i] < lower] = lower
        df[i][df[i] > upper] = upper
    return df

# Define all columns where outliers should be replaced
outlier_replace_cols = ['total_payment_plan_days', 'avg_payment_plan_days',
       'plan_net_worth', 'mean_payment_each_transaction',
       'total_actual_payment', 'average_discount', 'biggest_discount',
       'discount_times', 'amt_per_day', 'number_days_listened', 'num_25',
       'num_50', 'num_75', 'num_985', 'num_100', 'total_secs',
       'total_plays', 'percent_25', 'percent_50', 'percent_100',
       'avg_seconds_per_day', 'avg_plays_per_day', 'avg_unique_per_day',
       'day_diff_membership_expire__last_listen',
       'day_diff_last_listen__first_listen',
       'day_diff_membership_expire__registration']

# Apply the outlier replacement function on the data
dataset_train_v2 = replaceoutlier(dataset_train_v2, outlier_replace_cols)

<ipython-input-7-a828b365fddf>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][df[i] < lower] = lower
<ipython-input-7-a828b365fddf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][df[i] > upper] = upper


In [8]:
# Define the columns in which we are replacing NA's, along with the value which they should be replaced with
fill_na_cols = {'transaction_count':0, 'number_days_listened':0, 'num_25':0,
       'num_50':0, 'num_75':0, 'num_985':0, 'num_100':0, 'total_secs':0,
       'total_plays':0, 'avg_seconds_per_day':0, 'avg_plays_per_day':0, 'avg_unique_per_day':0}

dataset_train_v2 = dataset_train_v2.fillna(fill_na_cols)

In [9]:
# Here categorical variables with high cardinality are replaced with the mean of the target variable
# As we have not seperated test and train this will result in a small amount of leakage, which we will concee

# Define the columns where we will replace the categorical values with the target mean
target_mean_cols = ['city', 'normal_payment_method_id']

values = []
for i in target_mean_cols:
    # Take the mean value of the outcome variable (churn) for each categorical value
    df = dataset_train_v2.groupby(i)['is_churn'].mean().reset_index()
    df["feature"] = i
    list_means = df.to_numpy().tolist()
    # Create a list with the means and the categorical values
    for means_item in list_means:
        values.append(means_item)

# Create a data frame of mean values for each feature and value
means_df = pd.DataFrame(values, columns=['value', 'mean', 'feature'])

# Create new columns with the mean target mean encdings, and drop the old columns
for i in target_mean_cols:
    means = means_df[means_df.feature == i]
    means=means.rename(columns = {"value": i, "mean": i+"_mean"}).drop(["feature"], axis = 1)
    dataset_train_v2 = dataset_train_v2.merge(means, on = i, how = 'left')
    dataset_train_v2=dataset_train_v2.drop(i, axis = 1)
    

In [10]:
# Categorical values with low cardinality are one hot coded
ohe_cols = ['gender', 'registered_via']
dataset_train_v2 = pd.get_dummies(data = dataset_train_v2,columns = ohe_cols)


In [11]:
# Rename the churn column
dataset_train_v2=dataset_train_v2.rename(columns={"is_churn": "Churn"})

# Write the prepared data to the output location
path_new = "C:/Users/andrew.davidson/OneDrive - Concentra Consulting Limited/Documents/Projects/GBT Feature Relationships/KK Box/data/"
dataset_train_v2.to_csv(path_new + "Prepped Data.csv", index = False)